In [1]:
from preprocess import dataset,data_deal
from sklearn.preprocessing import MinMaxScaler
from models import *
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import KFold
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd


def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Config

In [2]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/total 12 data.xlsx'
data = pd.read_excel(file,engine="openpyxl")
data = np.array(data)
l = len(data)
para_path = './save/parameter'
batch_size = 300
plot = True
# np.set_printoptions(threshold=np.sys.maxsize)

# Train

In [3]:
def complexity(model):
    param_squared_sums = {}
    encoder_parameters = list(filter(lambda x: x[0].startswith('encoder'),list(model.named_parameters())))
    for name, param in encoder_parameters:
#         print(param)
        if 'weight' in name:
            param_squared_sums[name] = torch.sum(param ** 2)
    return sum(param_squared_sums.values())

In [4]:
total_loss = 0
flag =  3
Type = 'odd-even'
N_size = '<=126'
lr = 0.1
k = 10
alpha = 0.5
input_size = 12
epsilon = 0
seed1,seed2 = 0,0

In [5]:
kf = KFold(n_splits=k ,shuffle=True,random_state=seed1)

total_data = dataset(data,Type,N_size,input_size)
total_data = np.array(total_data,dtype = object)

X = total_data[:,0]
y = total_data[:,1].astype(np.float32)

temp_loss = 0
l1= X[0].shape[0]
l2 =len(X)
results_df = pd.DataFrame()
total_loss1 = 0
total_loss2 = 0

In [6]:
for fold, (train_index, test_index) in enumerate(kf.split(X)):
#     same_seeds(seed2)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    l3 = len(y_test)
    ae_model = get_models("AE_MLP",input_size,epsilon)
    mlp_model = get_models("MLP",input_size,epsilon)
    print(f"ae init weight {complexity(ae_model)}")
    print(f"mlp init weight {complexity(mlp_model)}")
    scaler = MinMaxScaler()
    X_train,X_test,y_train,y_test,scaler = data_deal(X_train,X_test,y_train,y_test,scaler,input_size,flag)
    same_seeds(seed2)
    train_MLP(X_train,y_train,ae_model,lr,alpha,fold,para_path,flag)
    same_seeds(seed2)
    loss,_ = test_MLP(X_test,y_test,ae_model,fold,para_path,scaler,results_df,l1,flag)
    total_loss1 = total_loss1 +loss
    loss = loss / l3
    print(f"ae test loss:{loss.item()}")
    same_seeds(seed2)
    train_MLP(X_train,y_train,mlp_model,lr,alpha,fold,para_path,flag)
    same_seeds(seed2)
    loss,results_df = test_MLP(X_test,y_test,mlp_model,fold,para_path,scaler,results_df,l1,flag)
    total_loss2 = total_loss2 +loss
    loss = loss / l3
    print(f"mlp test loss:{loss.item()}")
    print(f"ae trained weight {complexity(ae_model)}")
    print(f"mlp trained weight {complexity(mlp_model)}\n")
    
results_df.to_excel(f'test_data_and_outputs_{flag + 1}.xlsx', index=False)
print((sum(total_loss1)/l2)**0.5,(sum(total_loss2)/l2)**0.5)

ae init weight 26.09284782409668
mlp init weight 26.050159454345703
ae test loss:0.045533765107393265
mlp test loss:0.035664938390254974
ae trained weight 55.252437591552734
mlp trained weight 130.19735717773438

ae init weight 27.000120162963867
mlp init weight 25.392383575439453
ae test loss:0.03448549285531044
mlp test loss:0.023873208090662956
ae trained weight 55.061004638671875
mlp trained weight 145.17710876464844

ae init weight 27.000120162963867
mlp init weight 25.392383575439453
ae test loss:0.028917694464325905
mlp test loss:0.026189304888248444
ae trained weight 78.40351104736328
mlp trained weight 140.8511505126953

ae init weight 27.000120162963867
mlp init weight 25.392383575439453
ae test loss:0.10799577832221985
mlp test loss:0.12747976183891296
ae trained weight 43.286380767822266
mlp trained weight 145.49392700195312

ae init weight 27.000120162963867
mlp init weight 25.392383575439453
ae test loss:0.040041733533144
mlp test loss:0.032169077545404434
ae trained weig